In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import keras as K
import tensorflow as tf
import os, json
from scipy import misc
import cv2

Using TensorFlow backend.


In [1]:
from cnn_builder import *

Using TensorFlow backend.


In [2]:
m = get_NVIDIA()

In [2]:
IMAGE_DATA_PATH = './data'

In [3]:
image_data = pd.read_csv(os.path.join(IMAGE_DATA_PATH, 'driving_log.csv'))

In [4]:
image_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036 entries, 0 to 8035
Data columns (total 7 columns):
center      8036 non-null object
left        8036 non-null object
right       8036 non-null object
steering    8036 non-null float64
throttle    8036 non-null float64
brake       8036 non-null float64
speed       8036 non-null float64
dtypes: float64(4), object(3)
memory usage: 439.5+ KB


In [5]:
image_data.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [6]:
len(image_data)

8036

In [7]:
from sklearn.utils import shuffle

In [9]:
from sklearn.model_selection import train_test_split
ind_train, ind_test = train_test_split(np.array(range(len(image_data))), train_size=0.8, random_state = 42)

In [10]:
train_df = image_data.loc[ind_train]
test_df = image_data.loc[ind_test]

In [11]:
from keras.applications import VGG16

In [12]:
from keras.layers import Flatten, Dense, Dropout, Lambda, Cropping2D
from keras.models import Sequential, Model

In [13]:
def get_model(name, input_shape = (160,320,3), crop_from_top=50, crop_from_bottom=20):
    if name == 'VGG16':
        bottom = Sequential()
        bottom.add(Lambda(lambda x: x/255. - 0.5, input_shape=input_shape))
        bottom.add(Cropping2D(cropping=((crop_from_top,crop_from_bottom), (0,0)), input_shape=input_shape))
        m_bottom_model = Model(bottom.input, bottom.output)
        initial_model = VGG16(weights="imagenet", include_top=False, input_shape=m_bottom_model.output.get_shape().as_list()[1:])(m_bottom_model.output)
        x = Flatten()(initial_model)
        x = Dense(256, activation='relu')(x)
        output = Dense(1)(x)
        model = Model(m_bottom_model.input, output)

        for layer in model.layers[3].layers:
            layer.trainable = False
    else:
        ValueError
        
    return model

In [14]:
model = get_model('VGG16')

In [15]:
model.compile(loss='mse', optimizer='adam')

In [16]:
def generator_df(samples_df_, source_path='data', data_columns = ['center'], batch_size=4):
# yields batches from dataframe samples_df: ['images', 'steering']
    samples_df = samples_df_.copy()
    num_samples = len(samples_df)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples_df)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples_df[offset:offset+batch_size]

            images = None
            angles = np.array([], dtype='float32')
            for i, batch_sample in batch_samples.iterrows():
                name = batch_sample[np.random.choice(data_columns, 1)].values[0]
                center_image = cv2.imread(os.path.join(source_path,name))
                center_angle = batch_sample['steering']
                if images is None:
                    images = center_image[np.newaxis]
                else:
                    images = np.vstack([images, center_image[np.newaxis]])
                angles = np.append(angles, center_angle)

            yield images, angles

In [17]:
train_gen = generator_df(train_df)
val_gen = generator_df(test_df)

In [18]:
history = model.fit_generator(train_gen, samples_per_epoch=len(train_df), validation_data=val_gen,
                              nb_val_samples=len(test_df), nb_epoch=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=6428, epochs=1, validation_steps=1608)`
  


Epoch 1/1
6428/6428 [==============================] - 1930s - loss: 0.0330 - val_loss: 0.0086


In [21]:
history.history

{'loss': [0.032982737247569061], 'val_loss': [0.0085685270588595382]}

In [23]:
model.save('model1.h5')